In [1]:
import pandas as pd

import requests

import plotly.express as px


def get_request(url, parameters = {}):
    return requests.get(url, params = parameters)

url1 = "https://api.census.gov/data/2018/abscbo"

params = {

            "get": "GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_PCT",

            "for": "us",

            "key": "ab3bba1c660863685ea46afadc062bea471c8a64"

        }

res = get_request(url1, params).json()

ownerNat = pd.DataFrame(res[1:], columns = res[0])

ownerNat

,GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_PCT,us
0,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,001,All owners of respondent firms,O02,YRACQBUS,CI03,2013 to 2017,2018,21939,29.9,1
1,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,001,All owners of respondent firms,O02,YRACQBUS,CM03,2018,2018,912,1.2,1
2,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,001,All owners of respondent firms,O02,YRACQBUS,CN,Don't know,2018,7441,10.1,1
3,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,001,All owners of respondent firms,O02,YRACQBUS,CO,Total reporting,2018,73427,100.0,1
4,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,001,All owners of respondent firms,O02,YRACQBUS,CP,Item not reported,2018,3106,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104617,0100000US,United States,99,Industries not classified,003,Male,001,All owners of respondent firms,00,All owners of respondent firms,001,All owners of respondent firms,O15,PRIORBUSTAT,DI3,Business was purchased by another company,2018,24,0.4,1
104618,0100000US,United States,99,Industries not classified,003,Male,001,All owners of respondent firms,00,All owners of respondent firms,001,All owners of respondent firms,O15,PRIORBUSTAT,DI4,Business was purchased by another individual,2018,17,0.3,1
104619,0100000US,United States,99,Industries not classified,003,Male,001,All owners of respondent firms,00,All owners of respondent firms,001,All owners of respondent firms,O15,PRIORBUSTAT,DI5,Other,2018,273,4.4,1
104620,0100000US,United States,99,Industries not classified,003,Male,001,All owners of respondent firms,00,All owners of respondent firms,001,All owners of respondent firms,O15,PRIORBUSTAT,DITR,Total reporting,2018,6221,100.0,1


In [2]:
ownerNat['OWNPDEMP'] = ownerNat['OWNPDEMP'].astype(int)

In [3]:
ownerNat['NAICS2017_LABEL'].value_counts()

Total for all sectors                                                       5808
Professional, scientific, and technical services                            5695
Manufacturing                                                               5483
Wholesale trade                                                             5416
Health care and social assistance                                           5413
Accommodation and food services                                             5397
Construction                                                                5367
Retail trade                                                                5338
Administrative and support and waste management and remediation services    5332
Transportation and warehousing                                              5307
Information                                                                 5245
Other services (except public administration)                               5222
Real estate and rental and l

In [4]:
#Shortening overly-long industry labels
ownerNat['NAICS2017_LABEL']=ownerNat['NAICS2017_LABEL'].replace({'Agriculture, forestry, fishing and hunting':'Agriculture/Forestry/Fishing/Hunting', 'Mining, quarrying, and oil and gas extraction':'Mining/Quarrying/Oil&Gas', 'Professional, scientific, and technical services':'Professional/Sci/Tech', 'Administrative and support and waste management and remediation services':"Admin/Support/Waste Mgmt/Remediation"})



In [5]:
#finding service disabled male veterans
MSerDisVetOwn = ownerNat[(ownerNat.OWNER_VET_LABEL == 'Veteran') & (ownerNat.OWNER_SEX_LABEL == 'Male') & (ownerNat.OWNCHAR == 'FG')]


In [6]:
#finding male non-service-disabled male veterans
MNonServDisVetOwn = ownerNat[(ownerNat.OWNER_VET_LABEL == 'Veteran') & (ownerNat.OWNER_SEX_LABEL == 'Male') & (ownerNat.OWNCHAR == 'FH')]

In [7]:
#finding male veterans in general
MVDisNonDis = ownerNat[(ownerNat.OWNCHAR == 'FH') | (ownerNat.OWNCHAR == 'FG')]
MVDisNonDis = MVDisNonDis[((MVDisNonDis.OWNER_VET_LABEL == 'Veteran') & (MVDisNonDis.OWNER_SEX_LABEL == 'Male'))]

In [8]:
#veterans who served active duty post-9/11 by race

Vet911 = ownerNat[(ownerNat.OWNER_VET_LABEL == 'Veteran') & (ownerNat.OWNCHAR_LABEL == "Served on post-9/11 active duty military service") 
& (ownerNat.NAICS2017_LABEL == 'Total for all sectors') & (ownerNat.OWNER_RACE_LABEL != "All owners of respondent firms") 
& (ownerNat.OWNER_RACE_LABEL != "Minority") & (ownerNat.OWNER_RACE_LABEL != "Nonminority")]
Vet911

,GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_PCT,us
5302,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,002,Veteran,O14,VETSPECIFIC,MN,Served on post-9/11 active duty military service,2018,1300,16.5,1
5309,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,50,American Indian and Alaska Native,002,Veteran,O14,VETSPECIFIC,MN,Served on post-9/11 active duty military service,2018,335,17.4,1
5316,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,60,Asian,002,Veteran,O14,VETSPECIFIC,MN,Served on post-9/11 active duty military service,2018,1096,18.4,1
5323,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,70,Native Hawaiian and Other Pacific Islander,002,Veteran,O14,VETSPECIFIC,MN,Served on post-9/11 active duty military service,2018,0,0.0,1
6953,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,30,White,002,Veteran,O14,VETSPECIFIC,MN,Served on post-9/11 active duty military service,2018,20934,7.3,1


In [9]:
#finding veterans who served active duty in 2018 by race

Act2018VetRace = ownerNat[(ownerNat.OWNER_VET_LABEL == 'Veteran') & (ownerNat.OWNCHAR_LABEL == "Served on active duty military service in reference year") 
& (ownerNat.NAICS2017_LABEL == 'Total for all sectors') & (ownerNat.OWNER_RACE_LABEL != "All owners of respondent firms") 
& (ownerNat.OWNER_RACE_LABEL != "Minority") & (ownerNat.OWNER_RACE_LABEL != "Nonminority")]
Act2018VetRace

,GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_PCT,us
5303,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,002,Veteran,O14,VETSPECIFIC,MOSY,Served on active duty military service in refe...,2018,0,0.0,1
5310,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,50,American Indian and Alaska Native,002,Veteran,O14,VETSPECIFIC,MOSY,Served on active duty military service in refe...,2018,0,0.0,1
5317,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,60,Asian,002,Veteran,O14,VETSPECIFIC,MOSY,Served on active duty military service in refe...,2018,93,1.6,1
6954,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,30,White,002,Veteran,O14,VETSPECIFIC,MOSY,Served on active duty military service in refe...,2018,1190,0.4,1


In [10]:
#removing Native Hawaiian row becasuse they had zero respondents

Vet911 = Vet911[Vet911.OWNER_RACE_LABEL != 'Native Hawaiian and Other Pacific Islander']
Vet911

,GEO_ID,NAME,NAICS2017,NAICS2017_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNER_ETH,OWNER_ETH_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_VET,OWNER_VET_LABEL,QDESC,QDESC_LABEL,OWNCHAR,OWNCHAR_LABEL,YEAR,OWNPDEMP,OWNPDEMP_PCT,us
5302,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,40,Black or African American,002,Veteran,O14,VETSPECIFIC,MN,Served on post-9/11 active duty military service,2018,1300,16.5,1
5309,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,50,American Indian and Alaska Native,002,Veteran,O14,VETSPECIFIC,MN,Served on post-9/11 active duty military service,2018,335,17.4,1
5316,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,60,Asian,002,Veteran,O14,VETSPECIFIC,MN,Served on post-9/11 active duty military service,2018,1096,18.4,1
6953,0100000US,United States,00,Total for all sectors,001,All owners of respondent firms,001,All owners of respondent firms,30,White,002,Veteran,O14,VETSPECIFIC,MN,Served on post-9/11 active duty military service,2018,20934,7.3,1


In [19]:
#selecting for service-disabled and non-service disabled veterans of all genders

VDisNonDis = ownerNat[(ownerNat.OWNCHAR == 'FH') | (ownerNat.OWNCHAR == 'FG')]
VDisNonDis = VDisNonDis[((VDisNonDis.OWNER_VET_LABEL == 'Veteran'))]

Visualization included below:

fig01 = Race of Veteran Business Owners who Served post-9/11 Active Duty, 2018 <br>
fig02 = Race of Veteran Business Owners who Served Active Duty in 2018 <br>
fig03 = Service-Disabled Male Veteran Business Owners by Industry with 1000+ respondents, 2018 <br>
fig04 = Non-Service-Disabled Male Veterans Business Owners by Industry with over 1000 respondents, 2018 <br>
fig05 = Veteran Business Owners by Gender, 2018 <br>

In [11]:

fig01 = px.pie(Vet911, values="OWNPDEMP", names = "OWNER_RACE_LABEL", title ='Race of Veteran Business Owners who Served post-9/11 Active Duty, 2018')


fig01.update_layout(showlegend=True, margin=dict(l=25,r=25, t=50, b=50), 
legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.01
), legend_title_text='Race')

fig01.update_traces(textinfo=('value + percent'))



fig01.write_image(r"VetOwnRacePost911Pie.png")
fig01.show()

In [12]:
fig02 = px.bar(Act2018VetRace[2:], x = 'OWNPDEMP',  y= 'OWNER_RACE_LABEL', title='Race of Veteran Business Owners who Served Active Duty in 2018', orientation="h", color_discrete_sequence =['olive'], labels = {"OWNPDEMP": "Number of Owners", "OWNER_RACE_LABEL": "Race"}, height=300, width=1000)


fig02.update_layout(showlegend=False, barmode='stack', yaxis={'categoryorder':'total ascending'}, margin=dict(l=20,r=20,t=40,b=40))



fig02.write_image(r"VetOwnRace2018.png")


fig02.show()

In [13]:
#filtering business owners to include only those with over a thousand respondents

MSerDisVetOwn = MSerDisVetOwn[MSerDisVetOwn.OWNPDEMP > 1000]

In [14]:
#filtering business owners to include only those with over a thousand respondents
MNonSerDis = MNonServDisVetOwn[MNonServDisVetOwn.OWNPDEMP > 1000]


In [15]:

fig03 = px.bar(MSerDisVetOwn[1:], x = 'NAICS2017_LABEL',  y= 'OWNPDEMP', title="Service-Disabled Male Veteran Business Owners by Industry with over 1000 respondents, 2018", color_discrete_sequence =['purple']*len(MSerDisVetOwn), labels = {"OWNPDEMP": "Number of Owners", "NAICS2017_LABEL": "Industry"}, height=750, width=1000)



fig03.update_layout(showlegend=False, barmode='stack', xaxis={'categoryorder':'total descending'}, margin=dict(l=20,r=20,t=40,b=40))

fig03.update_xaxes(tickangle=45)


fig03.write_image(r"VetOwnMSD1000plus.png")


fig03.show()

In [16]:
fig04 = px.bar(MNonSerDis[1:], x = 'NAICS2017_LABEL',  y= 'OWNPDEMP', title="Non-Service-Disabled Male Veterans Business Owners by Industry with over 1000 respondents, 2018", color_discrete_sequence =['darkgreen']*len(MNonServDisVetOwn),labels = {"OWNPDEMP": "Number of Owners", "NAICS2017_LABEL": "Industry"}, height=750, width=1000)




fig04.update_layout(showlegend=False, barmode='stack', xaxis={'categoryorder':'total descending'}, margin=dict(l=20,r=20,t=40,b=40))



fig04.update_xaxes(tickangle=45)
fig04.show()


fig04.write_image(r"VetOwnMNSD1000plus.png")

In [18]:
fig05 = px.bar(VDisNonDis[:6], x = 'OWNER_SEX_LABEL',  y= 'OWNPDEMP', color ='OWNCHAR_LABEL', title = 'Veteran Business Owners by Gender, 2018', color_discrete_sequence=["purple", "darkgreen"], height = 400, labels = {"OWNPDEMP": "Number of Owners", "OWNER_SEX_LABEL": "Gender"})

fig05.update_layout(barmode='group', xaxis={'categoryorder':'total descending'}, legend_title_text='Service Disability Status')

fig05.show()


fig05.write_image(r"VetOwnGen.png")